# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('task_dwt.npz') 

# Access the concatenated features from the .npz file
X_dwt = data['X_dwt']

# Close the loaded file
data.close()

#print the merged array
X_dwt

array([[[-1.06148503e-04, -1.06694601e-04, -1.07029483e-04, ...,
         -2.94874301e-07, -4.23105620e-07,  3.54435175e-07],
        [-3.84645605e-05, -3.71460922e-05, -3.56534521e-05, ...,
          1.41689751e-07,  4.31346974e-07, -9.31883259e-07],
        [-2.64800307e-05, -2.69086897e-05, -2.82668046e-05, ...,
         -5.39172928e-08, -3.62517679e-07,  1.03366437e-06],
        ...,
        [ 5.96396755e-05,  5.91926693e-05,  5.86861593e-05, ...,
          5.98356922e-08,  4.42667182e-08,  7.50121572e-08],
        [-2.79652708e-05, -2.81712055e-05, -2.97970491e-05, ...,
         -2.94630633e-08, -3.91273507e-07,  1.20317652e-06],
        [ 1.38124990e-05,  1.54131628e-05,  2.01976292e-05, ...,
          8.64259098e-07,  1.77865761e-06, -2.97942169e-06]],

       [[ 1.07158417e-04,  1.07484494e-04,  1.08244955e-04, ...,
         -1.15065306e-08, -7.16634965e-08,  1.48975747e-07],
        [-1.66651973e-05, -1.64979475e-05, -1.75584333e-05, ...,
         -8.01448781e-08, -3.12057334e

In [3]:
data = np.load('task_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('task_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_dwt.shape, Y.shape, groups_array.shape)

(38049, 20, 205) (38049,) (38049,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_dwt, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_dwt[train_index], X_dwt[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
134/134 [==============================] - 16s 41ms/step - loss: 0.3330 - accuracy: 0.8654 - val_loss: 0.3145 - val_accuracy: 0.8891
Epoch 2/200
134/134 [==============================] - 3s 24ms/step - loss: 0.2245 - accuracy: 0.9125 - val_loss: 0.2245 - val_accuracy: 0.9078
Epoch 3/200
134/134 [==============================] - 3s 23ms/step - loss: 0.1727 - accuracy: 0.9354 - val_loss: 0.1704 - val_accuracy: 0.9351
Epoch 4/200
134/134 [==============================] - 3s 23ms/step - loss: 0.1384 - accuracy: 0.9479 - val_loss: 0.1538 - val_accuracy: 0.9393
Epoch 5/200
134/134 [==============================] - 3s 25ms/step - loss: 0.1229 - accuracy: 0.9539 - val_loss: 0.1401 - val_accuracy: 0.9422
Epoch 6/200
134/134 [==============================] - 3s 24ms/step - loss: 0.1063 - accuracy: 0.9596 - val_loss: 0.1359 - val_accuracy: 0.9509
Epoch 7/200
134/134 [==============================] - 3s 23ms/step - loss: 0.0955 - accuracy: 0.9641 - val_loss: 0.1235 - val_a

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9753,0.9699,0.9857,0.9777,0.9741,0.9626,0.9857,0.9500,0.9502
1,0.9735,0.9624,0.9904,0.9762,0.9716,0.9527,0.9904,0.9462,0.9467
2,0.9798,0.9710,0.9928,0.9818,0.9783,0.9638,0.9928,0.9590,0.9593
3,0.9748,0.9677,0.9871,0.9773,0.9734,0.9597,0.9871,0.9489,0.9492
4,0.9708,0.9639,0.9837,0.9737,0.9694,0.9550,0.9837,0.9409,0.9412
5,0.9798,0.9706,0.9933,0.9818,0.9783,0.9632,0.9933,0.9590,0.9593
6,0.9779,0.9731,0.9871,0.9801,0.9769,0.9667,0.9871,0.9553,0.9555
7,0.9766,0.9639,0.9947,0.9791,0.9746,0.9544,0.9947,0.9526,0.9532
8,0.9771,0.9678,0.9914,0.9795,0.9755,0.9597,0.9914,0.9537,0.9540
9,0.9777,0.9700,0.9900,0.9799,0.9763,0.9626,0.9900,0.9548,0.9550


In [13]:
metrics_df.round(4).to_csv('DWT_TASK_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("DWT_TASK_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9753,0.9699,0.9857,0.9777,0.9741,0.9626,0.9857,0.9500,0.9502
1,0.9735,0.9624,0.9904,0.9762,0.9716,0.9527,0.9904,0.9462,0.9467
2,0.9798,0.9710,0.9928,0.9818,0.9783,0.9638,0.9928,0.9590,0.9593
3,0.9748,0.9677,0.9871,0.9773,0.9734,0.9597,0.9871,0.9489,0.9492
4,0.9708,0.9639,0.9837,0.9737,0.9694,0.9550,0.9837,0.9409,0.9412
5,0.9798,0.9706,0.9933,0.9818,0.9783,0.9632,0.9933,0.9590,0.9593
6,0.9779,0.9731,0.9871,0.9801,0.9769,0.9667,0.9871,0.9553,0.9555
7,0.9766,0.9639,0.9947,0.9791,0.9746,0.9544,0.9947,0.9526,0.9532
8,0.9771,0.9678,0.9914,0.9795,0.9755,0.9597,0.9914,0.9537,0.9540
9,0.9777,0.9700,0.9900,0.9799,0.9763,0.9626,0.9900,0.9548,0.9550


In [16]:
(result.mean()*100).round(2)

Accuracy       97.63
Precision      96.80
Recall         98.96
F1             97.87
Sensitivity    97.48
Specificity    96.00
ROC_AUC        98.96
Kappa          95.20
MCC            95.24
dtype: float64

In [17]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])